In [40]:
import pandas as pd
import nltk
import numpy

In [5]:
df = pd.read_csv('/Users/yetiancl/Dropbox/Kaggle/toxic/train.csv')
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [ ]:
## get locations of negation 

In [249]:
def find_string(wordlist,string):
    return [i for i, x in enumerate(wordlist) if x == string]

nt_loc = df['comment_text'].apply(lambda x: [find_string(nltk.word_tokenize(x),"n't")])

nt_loc = nt_loc.apply(lambda x: x[0] if len(x[0]) > 0 else [])

In [252]:
not_loc = df['comment_text'].apply(lambda x: [find_string(nltk.word_tokenize(x),"not")])
not_loc = not_loc.apply(lambda x: x[0] if len(x[0]) > 0 else [])

In [293]:
df = df.assign(neg_loc = nt_loc + not_loc)

## get locations of bad words and ethnic slurs

In [172]:
badword_list = pd.read_csv("../bad_word_list.csv",encoding="latin1",header=None)
badword_list = badword_list[0].tolist()

In [258]:
ethnic_slurs = pd.read_csv("../ethnic_slurs.csv",header=None)
ethnic_slurs = ethnic_slurs[0].tolist()

In [164]:
df['comment_list'] = df['comment_text'].apply(lambda x: nltk.word_tokenize(x))

In [226]:
df['has_bad'] = df['comment_list'].apply(lambda x: set(x).intersection(set(badword_list)))

In [223]:
def get_bad_loc(df):
    wordlist = df['comment_list']
    bad_loc = []
    if len(df['has_bad']) > 0: 
        for word in df['has_bad']:
            loc_this = find_string(wordlist,word)
            if len(loc_this) >0:  
                bad_loc = bad_loc + loc_this
    return bad_loc

df['bad_loc'] = df.apply(get_bad_loc, axis = 1)

In [290]:
df['has_ethnic'] = df['comment_list'].apply(lambda x: set(x).intersection(set(ethnic_slurs)))

In [289]:
ethnic = df['has_ethnic'].apply(lambda x: len(x) > 0)
ethnic.value_counts()

False    159319
True        252
Name: has_ethnic, dtype: int64

In [291]:
def get_ethnic_loc(df):
    wordlist = df['comment_list']
    ethnic_loc = []
    if len(df['has_ethnic']) > 0: 
        for word in df['has_ethnic']:
            loc_this = find_string(wordlist,word)
            if len(loc_this) >0:  
                ethnic_loc = ethnic_loc + loc_this
    return ethnic_loc

df['ethnic_loc'] = df.apply(get_ethnic_loc, axis = 1)

##  check if bad words and ethnic slurs were negated, if there is a neg word within 5 words before them

In [306]:
neg = [11, 30, 56]
shit = [13, 33]

In [310]:
def which_bad_negated(df):
    neg = df['neg_loc']
    shit = df['bad_loc']
    negated = []
    for i in shit:
        negated = negated + [x for x in neg if (i-5) < x < i]
    return negated

In [313]:
df['which_bad_negated'] = df.apply(which_bad_negated, axis = 1)

In [314]:
def which_ethnic_negated(df):
    neg = df['neg_loc']
    ethnic = df['ethnic_loc']
    negated = []
    for i in ethnic:
        negated = negated + [x for x in neg if (i-5) < x < i]
    return negated

df['which_ethnic_negated'] = df.apply(which_ethnic_negated, axis = 1)

In [ ]:
## get location of "you"